# Introduction to functional programming in Python
<table style="margin-top:30px; border:none" align="left">
    <tr style="border:none"><td style="border:none"><p>Francesco Bruni</p></td></tr>
    <tr style="border:none" style="width:100%">
        <td style="border:none" style="width:90%"><p>@brunifrancesco</p></td>
    </tr>
</table>


##Who am I

- MSc Student in Telecommunication Engineering @ Poliba
- Despite a Java background, I prefer Python whenever possible
- I'm not a computer scientist

##What we'll talk about

- Why functional programming
- Why Python
- Functional features in Python
    - First class functions
    - Immutable vs mutable data
    - Lazy evaluation
    - Recursion vs loop
    - (Manual) Currying
- Useful functional programming resources
- Conclusions


#Why functional programming
- Think in terms of functions
- State as function evaluation, not variables assignment
- Testing is (more) easy
- A new viewpoint is required

In [1]:
import random
def sum_imperative():
    res = 0
    for n in [random.random() for _ in range(100)]:
        res += n
    return res

def sum_functional():
    return sum(random.random() for _ in range(100))


#Why Python

There are a lot of languages out there..

##Pros
- Mature enough
- Multiparadigm and multipurpose
- Easy to learn
- Interactive shell
- Multiple implementantions (C, Java, Python, .NET, Ruby, Js)
- Extending requires few lines of code



##Cons
- Python 3 vs Python 2
- No typed variables
- CPython is the widest used implementation
- Changing implementation (or version upgrading) is not (always) a free iussues transition

##Python is not Java
**Task**: generate a list of numbers between 1 and 100, multiple of 2, whose does not end with the "4" digit


In [3]:
def java_approach(size=100):
    l = []
    i=10
    while i < size:
        if i % 2 == 0 and not str(i).endswith("4"):
          l.append(i)
        i += 1
    return l

In [4]:
def python_better_approach(size=100):
    l = [item for item in range(10, size) if not item % 2 and not str(item).endswith("4")]
    return l

assert(java_approach() == python_better_approach())

##Python signatures supports Java **varags** (and much more)

In [5]:
def python_better_approach(size, *args, **kwargs):
    l = [item for item in range(10, size) if not item % 2 and not str(item).endswith("4")]
    return l

assert(java_approach() == python_better_approach(**dict(size=100, ciccio="wow!")))
assert(java_approach() == python_better_approach(size=100))

In [6]:
class SampleData(object):
    def __init__(self, x, y, z, t):
        self.x = x
        self.y = y
        self.z = z
        self.t = t
    
    def __eq__(self, other):
        return (True if self.__dict__ == other.__dict__ else False)

sample_data_1 = SampleData(x=1,y=2,z=3,t=4)
sample_data_2 = SampleData(**dict(x=1,y=2,z=3,t=4))

assert(sample_data_1 == sample_data_2)

#Functional features in Python

- Not all functional patterns apply to Python
- Hybrid approach is often requested
- Other libraries needs to be (eventually) integrated



##First class functions

- Functions are objects too
- They have fields too and some basic method


In [15]:
def func_1(x="32"):
    assert(x)

def func_2():
    return "wow!"

func_3 = lambda some_param: str(some_param)

##High order functions

- Functions which accept functions as params;
- Functions which return other functions;
- Python std lib examples for mapping: **filter**, **map**, **sorted**, 
- Python std lib examples for reducing: **max**, **min**, **sum**

**Task**: given the previous generated list, filter some number out and sort the result by the second digit.

In [11]:
def filter_out():
    
    result = python_better_approach()
    
    exclude = map(lambda item: item ** 2, range(30))

    result = filter(lambda item: item not in exclude, result)
    
    return sorted(result, key=lambda item: str(item)[1])
    

filter(lambda item: item not in map(lambda item: item ** 2, range(30)), python_better_approach(size=100))

##Class for function composition
Handle class creation via a new class syntax, allowing a "static" configuration


In [54]:
from collections.abc import Callable

class ExcludeFunction(Callable):
    def __init__(self, exclude_function):
        self.exclude_function= exclude_function
    
    def __call__(self, value):
        return None if not value else self.exclude_function(value)

result = python_better_approach(size=100)
exclude_function_1 = lambda item: item ** 2
ex_func = ExcludeFunction(exclude_function_1)

result = filter(lambda item: not item == ex_func(item), result)
assert(sorted(result, key=lambda item: str(item)[1]) == filter_out())

#Immutable vs mutable data structure

- Since state is not given by variable assignement, there's no need of mutables data
- (Almost) no classes are required
- Instead of lists, **tuples** are reccomanded
- Instead of classes, **namedtuple** can be used

In [1]:
import random
import pprint
from collections import namedtuple

MyObj = namedtuple("MyClassReplacement",("some_string", "my_custom_function"))
o = MyObj(some_string=str(random.random() + 4), my_custom_function=lambda item: float(item)*3)
assert(o.my_custom_function(o.some_string) == float(o.some_string) * 3)


r = [(MyObj(some_string=str(random.random() + 4), my_custom_function=lambda item: float(item)*3), _) for _ in range(3)]

def nested_tuples():
    r_1 = [(i, index) for index, i in enumerate(r)]
    
def flat_tuples():
    r_2 = [(i[0], i[1], index) for index, i in enumerate(r)]


##Strict vs not strict evaluation

- Strict evaluation requires that all operators needs to be evaluated
- Non strict (or lazy) evaluation, evaluates expression if and when requested

In [18]:
assert((python_better_approach(size=100) or 0))
assert(not(0 and python_better_approach(size=100)))

##Use case: Python iterables structures

- **Iterable**: objects than can be iterated;
- **Iterator**: objects than can be iterated and consumed only once, with two main problems:
    - They track their state in a stateful object
    - Their values are generated *a priori*
    - They aren't lazy
- **Generators**: lazy evaluated data structures:
    - They track state in function, instead of object
    - They don't act as normal functions
    - They generate next member when invoked

In [74]:
def _iter(size=100):
    return iter(python_better_approach(size=size))


def lazy_python_approach(size=100):
     for item in range(10, size):
        if not item % 2 and not str(item).endswith("4"):
            yield item

def lazy_new_python_approach(size=100):
    yield from (r for r in range(10, size) if not r % 2 and not str(r).endswith("4"))

##Recursion vs loop
- Functional programming requires a recursion approach vs loop iteration
- Python suffers by recursion limit
- Python does not offer any tail call optimization

In [7]:
def facti(n):
    if n == 0: return 1
    f= 1
    for i in range(2,n):
        f *= i
    return f

def fact(n):
    if n == 0: return 1
    else: return n*fact(n-1)

##Currying

Multiple arguments functions mapped to single arguments functions

In [19]:
from random import randrange

def mult(a, b, c ):
    def wrapper_1(b):
        def wrapper_2(c):
            return a*b*c
        return wrapper_2(c)
    return wrapper_1(b)


def mult_2(a):
    def wrapper_1(b):
        def wrapper_2(c):
            return a*b*c
        return wrapper_2
    return wrapper_1


assert(mult(2,4,8) == 64)
assert(mult_2(2)(3)(4) == 24)

##Partials
Python provides "partial" functions for manual currying

In [21]:
from functools import reduce
import operator
import random

def two_random_sum():
    return reduce(operator.sum, [random.random()]*2)

def three_random_product():
    return reduce(operator.mul, [random.random()]*3)

def four_random_sub():
    return reduce(operator.sub, [random.random()]*4)

def five_random_pow ():
    return reduce(operator.pow, [random.random()]*5)


In [24]:
def handle_random_numbers(size, function):
    return reduce(function, [random.random()]*size)

two_random_sum = partial(handle_random_numbers, size=2)
three_random_sum = partial(handle_random_numbers, size=3)

two_random_pow = partial(handle_random_numbers, size=2, function=operator.pow)
five_random_product = partial(handle_random_numbers, size=5, function=operator.mul)


##Decorator Pattern

Return a modified version of a decorated function

In [116]:
from functools import wraps
from functools import partial

def get_ned_data(n):
    def get_doubled_data(func, *args, **kwargs):
        @wraps(func)
        def _inner(*args, **kwargs):
            kwargs["new_param"] = kwargs["some_param"]*n
            return func(*args, **kwargs)
        return _inner
    return get_doubled_data


@get_ned_data(n=2)
def double_func(*args, **kwargs):
    assert(kwargs["new_param"] == kwargs["some_param"]*2)

@get_ned_data(n=3)
def triple_func(*args, **kwargs):
    assert(kwargs["new_param"] == kwargs["some_param"]*3)
    
double_func(some_param=3)
triple_func(some_param=5)

#Monads


- How to handle error in functional programming?
- Practical example: parsing and tranforming data coming from http request
- Python "std" libs have no support

In [20]:
from fn.monad import optionable

def get(request, *args, **kwargs):
    @optionable
    def _get_values(data):
        return data.get("values", None)
    _split = lambda item: item.split(",")
    _strip = lambda item: item.replace(" ", "")
    _filter = lambda item: list(filter(lambda i: i, item))
    values = _get_values(request.GET).map(_strip).map(_split).map(_filter).get_or(["v1,v2"])
    return values


from collections import namedtuple
def test():
    _req_class = namedtuple("Request", ("GET"))
    
    request_1 = _req_class(dict(values="v1, v2,v3"))
    request_2 = _req_class(dict(values="v1,v2,v3 "))
    request_3 = _req_class(dict(values="v1, v2,v3, "))
    
    assert(get(request_1) == ['v1', 'v2', 'v3'])
    assert(get(request_2) == ['v1', 'v2', 'v3'])
    assert(get(request_3) == ['v1', 'v2', 'v3'])

test()

 #Useful libraries

- **Fn.py** (https://github.com/kachayev/fn.py) (Scala functional features ported to Python)
- **Underscore.py** (ported from underscore_js)

#Conclusion
- Functional programming for writing more succint code
- Change viewpoint: think in terms of function
- Python seems for dummies until you show some cool features
- Python can be slow but with a fine tuning can be more efficien

#Questions?